# Post-process vasca_pipe results
Make catalog file of selected sources only, associate and get store additional info for listed sources

In [1]:
region_name = "ALL_10-800_LOOSE" #"CAINGSGII_10-800"  #"TDS" #"CAINGSGII_10-800"  #"TDS" # "MDIS_10-800" # "TDS" # _ELAISN1
region_fname = "./vasca_pipeline/"+region_name+"/region_"+region_name+".fits"
region_cat_fname = region_fname.replace(".fits","_cat.fits")

do_cat = False                        # Recreate catalog file from vasca_pipe output region
do_cross_match = False                  # Does source matching with CDS
do_src_prep = True                  # Gets additional info for sources listes in srcs_ids list below.
do_variable_addition = False            # Adds variables of matched sources to tt_sources
do_LombScargle = False

srcs_ids = []
srcs_ids = [4993 ,13580 ,14504 ,30475 ,50388 ,52994 ,79477 ,120265 ,138783 ,139480 ,162697 ,204419 ,221349 ,228958 ,236839 ,253214 ,253670
            ,256433 ,261213 ,331466 ,340464 ,381336 ,390864 ,489430 ,498065 ,525310 ,532554 ,533185 ,540235 ,583838 ,639720 ,645215 ,645730
            ,659787 ,666224 ,673175 ,680119 ,691408 ,714140 ,764827 ,1037284 ,1069077 ,1070938 ,1805505 ,1839206 ,2527679 ,3074917 ,] # WD ALL_10-800-LOOSE

In [2]:
%matplotlib widget
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np

from astropy.coordinates import SkyCoord
from astropy.table import unique, Table, hstack, join, setdiff
from astropy import units as uu
from astroquery.simbad import Simbad
from astropy.visualization import quantity_support
from astropy.modeling.models import BlackBody
from astropy import constants as cc
from matplotlib.ticker import ScalarFormatter
from astroquery.sdss import SDSS

from vasca.region import Region
from vasca.field import BaseField
import vasca.visualization as vvis
from vasca.tables import TableCollection, dd_vasca_columns
from vasca.utils import otype2ogroup,dd_ogrp2otypes, dd_ogrp2col, dd_filter2wavelength, mag2flux, flux2mag, query_vizier_sed, add_ogrp

from loguru import logger
logger.enable("vasca")

# Do catalog file

if do_cat:
    rg = Region()
    rg.load_from_fits(region_fname) 
    rc = rg.get_region_catalog()
else:
    rc = Region()
    rc.load_from_fits(region_cat_fname)

#Cross match to SIMBAD and GAIA
if do_cross_match:
    rc.cross_match_cds(query_table="simbad")
    rc.cross_match_cds()

#Add variables
if do_variable_addition:
    rc.add_column("tt_gaiadr3", "Gmag_abs")
    sel_plx = rc.tt_gaiadr3["RPlx"]>3
    rc.tt_gaiadr3["Gmag_abs"][sel_plx]=rc.tt_gaiadr3["Gmag"][sel_plx] + 5.0 * np.log10(rc.tt_gaiadr3["Plx"][sel_plx]) - 10.0

    # Add object groups
    add_ogrp(rc.tt_simbad)
    add_ogrp(rc.tt_gaiadr3,provenance="GAIA")

if do_LombScargle:
    rc.set_LombScargle()

#Write out all changes to region
rc.write_to_fits(region_cat_fname, overwrite=True)

#Prepare sources, including Vizier SED.
if do_src_prep:
    for src_id in srcs_ids:
        tc_src = rc.get_src_from_id(src_id, load_from_file = False)
print("Done")

2023-10-27 13:14:42.243 | DEBUG    | vasca.tables:load_from_fits:330 - Loading file with name './vasca_pipeline/ALL_10-800_LOOSE/region_ALL_10-800_LOOSE_cat.fits'
2023-10-27 13:14:42.250 | DEBUG    | vasca.tables:load_from_fits:341 - Loading table 'tt_fields'
2023-10-27 13:14:42.261 | DEBUG    | vasca.tables:load_from_fits:341 - Loading table 'tt_filters'
2023-10-27 13:14:42.267 | DEBUG    | vasca.tables:load_from_fits:341 - Loading table 'tt_visits'
2023-10-27 13:14:42.274 | DEBUG    | vasca.tables:load_from_fits:341 - Loading table 'tt_sources'
2023-10-27 13:14:42.292 | DEBUG    | vasca.tables:load_from_fits:341 - Loading table 'tt_detections'
2023-10-27 13:14:42.325 | DEBUG    | vasca.tables:load_from_fits:341 - Loading table 'tt_coadd_detections'
2023-10-27 13:14:42.400 | DEBUG    | vasca.tables:load_from_fits:341 - Loading table 'tt_coadd_sources'
2023-10-27 13:14:42.417 | DEBUG    | vasca.tables:load_from_fits:341 - Loading table 'tt_src_id_map'
2023-10-27 13:14:42.424 | DEBUG   

Nr of spectra found 1


2023-10-27 13:15:23.895 | DEBUG    | vasca.tables:write_to_fits:264 - Writing table 'tt_lombscargle'
2023-10-27 13:15:23.902 | DEBUG    | vasca.tables:write_to_fits:264 - Writing table 'tt_visits'
2023-10-27 13:15:23.908 | DEBUG    | vasca.tables:write_to_fits:264 - Writing table 'tt_fields'
2023-10-27 13:15:23.920 | DEBUG    | vasca.tables:write_to_fits:264 - Writing table 'tt_filters'
2023-10-27 13:15:23.926 | DEBUG    | vasca.tables:write_to_fits:264 - Writing table 'tt_source_lc'
2023-10-27 13:15:23.937 | DEBUG    | vasca.tables:write_to_fits:264 - Writing table 'tt_sed'
2023-10-27 13:15:23.942 | DEBUG    | vasca.tables:write_to_fits:264 - Writing table 'tt_gphoton_lc'
2023-10-27 13:15:23.951 | DEBUG    | vasca.tables:write_to_fits:264 - Writing table 'tt_spectrum'
2023-10-27 13:15:24.088 | DEBUG    | vasca.region:get_src_from_id:338 - Getting Source object for rg_src_id: 14504
2023-10-27 13:15:24.090 | DEBUG    | vasca.tables:add_table:159 - Adding table 'tt_sources'
2023-10-27 13

Nr of spectra found 1


2023-10-27 13:15:41.448 | DEBUG    | vasca.tables:write_to_fits:264 - Writing table 'tt_lombscargle'
2023-10-27 13:15:41.455 | DEBUG    | vasca.tables:write_to_fits:264 - Writing table 'tt_visits'
2023-10-27 13:15:41.460 | DEBUG    | vasca.tables:write_to_fits:264 - Writing table 'tt_fields'
2023-10-27 13:15:41.471 | DEBUG    | vasca.tables:write_to_fits:264 - Writing table 'tt_filters'
2023-10-27 13:15:41.476 | DEBUG    | vasca.tables:write_to_fits:264 - Writing table 'tt_source_lc'
2023-10-27 13:15:41.485 | DEBUG    | vasca.tables:write_to_fits:264 - Writing table 'tt_sed'
2023-10-27 13:15:41.491 | DEBUG    | vasca.tables:write_to_fits:264 - Writing table 'tt_spectrum'
2023-10-27 13:15:41.613 | DEBUG    | vasca.region:get_src_from_id:338 - Getting Source object for rg_src_id: 30475
2023-10-27 13:15:41.616 | DEBUG    | vasca.tables:add_table:159 - Adding table 'tt_sources'
2023-10-27 13:15:41.640 | DEBUG    | vasca.tables:add_table:159 - Adding table 'tt_detections'
2023-10-27 13:15:4

...

Nr of spectra found 1


2023-10-27 13:18:49.257 | DEBUG    | vasca.tables:write_to_fits:264 - Writing table 'tt_visits'
2023-10-27 13:18:49.263 | DEBUG    | vasca.tables:write_to_fits:264 - Writing table 'tt_fields'
2023-10-27 13:18:49.274 | DEBUG    | vasca.tables:write_to_fits:264 - Writing table 'tt_filters'
2023-10-27 13:18:49.278 | DEBUG    | vasca.tables:write_to_fits:264 - Writing table 'tt_source_lc'
2023-10-27 13:18:49.288 | DEBUG    | vasca.tables:write_to_fits:264 - Writing table 'tt_sed'
2023-10-27 13:18:49.295 | DEBUG    | vasca.tables:write_to_fits:264 - Writing table 'tt_gphoton_lc'
2023-10-27 13:18:49.304 | DEBUG    | vasca.tables:write_to_fits:264 - Writing table 'tt_spectrum'
2023-10-27 13:18:49.433 | DEBUG    | vasca.region:get_src_from_id:338 - Getting Source object for rg_src_id: 236839
2023-10-27 13:18:49.437 | DEBUG    | vasca.tables:add_table:159 - Adding table 'tt_sources'
2023-10-27 13:18:49.462 | DEBUG    | vasca.tables:add_table:159 - Adding table 'tt_detections'
2023-10-27 13:18:4

Nr of spectra found 2


2023-10-27 13:26:59.589 | DEBUG    | vasca.region:get_src_from_id:338 - Getting Source object for rg_src_id: 1839206
2023-10-27 13:26:59.595 | DEBUG    | vasca.tables:add_table:159 - Adding table 'tt_sources'
2023-10-27 13:26:59.626 | DEBUG    | vasca.tables:add_table:159 - Adding table 'tt_detections'
2023-10-27 13:26:59.637 | DEBUG    | vasca.tables:add_table:159 - Adding table 'tt_simbad'
2023-10-27 13:26:59.657 | DEBUG    | vasca.tables:add_table:159 - Adding table 'tt_gaiadr3'
2023-10-27 13:26:59.674 | DEBUG    | vasca.tables:add_table:159 - Adding table 'tt_visits'
2023-10-27 13:26:59.680 | DEBUG    | vasca.tables:add_table:159 - Adding table 'tt_fields'
2023-10-27 13:26:59.682 | DEBUG    | vasca.tables:add_table:159 - Adding table 'tt_filters'
2023-10-27 13:26:59.703 | DEBUG    | vasca.tables:add_table:159 - Adding table 'tt_source_lc'
2023-10-27 13:27:19.585 | DEBUG    | vasca.tables:add_table:159 - Adding table 'region:tt_sed'
2023-10-27 13:27:19.605 | DEBUG    | vasca.tables:

Done
